In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [2]:
MOVIE_NAME = []
MOVIE_DESCRIPTION = []
MOVIE_RATINGS = []
CAST_DETAILS = []
DIRECTOR_DETAILS = []
MOVIE_BUDGET = []
MOVIE_COLLECTION = []
VERDICT = []

In [3]:
MOVIE_NAME = []

MOVIE_DESCRIPTION = []
MOVIE_RATINGS = []
CAST_DETAILS = []
FILTERED_CAST = []
DIRECTOR_DETAILS = []
MOVIE_BUDGET = []
MOVIE_COLLECTION = []
VERDICT = []
movie_urls = ["https://www.imdb.com/list/ls536865476/","https://www.imdb.com/list/ls514198042/","https://www.imdb.com/list/ls536991867/","https://www.imdb.com/list/ls536998572/","https://www.imdb.com/list/ls043581860/","https://www.imdb.com/list/ls041278879/","https://www.imdb.com/list/ls513706508/","https://www.imdb.com/list/ls516348648/","https://www.imdb.com/list/ls526404520/?year=1980%2C2024","https://www.imdb.com/list/ls562330283/","https://www.imdb.com/list/ls041278879/"]
total_urls = len(movie_urls)
n = 1
for urls in movie_urls:
    print(f"Opening {n}/{total_urls} urls")
    driver_path = "chromedriver.exe"
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    url = f"{urls}"
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    info = soup.find("ul", class_="ipc-metadata-list ipc-metadata-list--dividers-between sc-e22973a9-0 khSCXM detailed-list-view ipc-metadata-list--base")
    movies_collection = info.find_all("div", class_="sc-e2db8066-1 QxXCO dli-parent")
    for movie_details in movies_collection:
        movie_name = movie_details.find("h3", class_="ipc-title__text")
        movie_name = movie_name.text.strip()
        movie_name_cleaned = re.sub(r'^\d+\.\s+', '', movie_name, flags=re.MULTILINE)
        movie_description = movie_details.find("div", class_="ipc-html-content-inner-div")
        movie_rating = movie_details.find("span", class_="ipc-rating-star--rating")
        cast_details = movie_details.find_all("a", class_="ipc-link ipc-link--base sttd-cast-item")
        for cast in cast_details:
            CAST_DETAILS.append(cast.text)
        arranged_cast = []
        for i in range(0,len(CAST_DETAILS),3):
            arranged_cast.append(CAST_DETAILS[i:i+3])
        directors_name = movie_details.find("a", class_="ipc-link ipc-link--base sttd-director-item")
        movies_collection = movie_details.find_all("div", class_="ipc-html-content-inner-div")
        for movie_collection in movies_collection:
            collection = movie_collection.text
            for line in collection.split('\n'):
                if "Worldwide Collection" in line:
                    movie_budget = line.split(":")[3].strip()
                    world_wide_collection = line.split(":")[5].strip()
                    movie_verdict = line.split(":")[-1].strip()
                    verdict_splitted = movie_verdict.split("•")
                    MOVIE_COLLECTION.append(world_wide_collection)
                    MOVIE_BUDGET.append(movie_budget)
                    VERDICT.append(verdict_splitted[0])
        MOVIE_NAME.append(movie_name_cleaned)
        MOVIE_DESCRIPTION.append(movie_description.text)
        MOVIE_RATINGS.append(movie_rating.text)
        DIRECTOR_DETAILS.append(directors_name.text)
        FILTERED_CAST.append(arranged_cast[-1])
    driver.quit()
    n+=1
MOVIE_NAME = MOVIE_NAME[:673]
MOVIE_DESCRIPTION = MOVIE_DESCRIPTION[:673]
MOVIE_RATINGS = MOVIE_RATINGS[:673]
FILTERED_CAST = FILTERED_CAST[:673]
DIRECTOR_DETAILS = DIRECTOR_DETAILS[:673]
print(len(MOVIE_NAME), len(MOVIE_DESCRIPTION), len(MOVIE_RATINGS), len(FILTERED_CAST), len(DIRECTOR_DETAILS), len(MOVIE_BUDGET), len(MOVIE_COLLECTION),len(VERDICT))


Opening 1/11 urls
Opening 2/11 urls
Opening 3/11 urls
Opening 4/11 urls
Opening 5/11 urls
Opening 6/11 urls
Opening 7/11 urls
Opening 8/11 urls
Opening 9/11 urls
Opening 10/11 urls
Opening 11/11 urls
673 673 673 673 673 673 673 673


In [4]:
CAST_1 = []
CAST_2 = []
CAST_3 = []

for cast in FILTERED_CAST:
    CAST_1.append(cast[0])
    CAST_2.append(cast[1])
    CAST_3.append(cast[-1])
print(len(CAST_1), len(CAST_2), len(CAST_3))

673 673 673


In [6]:
data = {
    'Movie Name': MOVIE_NAME,
    'Movie Description': MOVIE_DESCRIPTION,
    'Movie Rating': MOVIE_RATINGS,
    'Cast_1': CAST_1,
    'Cast_2': CAST_2,
    'Cast_3': CAST_3,
    'Director': DIRECTOR_DETAILS,
    'Budget': MOVIE_BUDGET,
    'Collection': MOVIE_COLLECTION,
    'Verdict': VERDICT
}

df = pd.DataFrame(data)
df['Budget'] = df['Budget'].apply(lambda x: re.findall('(\d+)[.]',x))
df['Collection'] = df['Collection'].apply(lambda x: re.findall('(\d+)[.]',x))
df['Budget'] = df['Budget'].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') else x)
df['Budget'] = df['Budget'].apply(lambda x: x[0] if is=instance(x, list) and x else None)
df['Collection'] = df['Collection'].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') else x)
df['Collection'] = df['Collection'].apply(lambda x: x[0] if isinstance(x, list) and x else None)
df

,Movie Name,Movie Description,Movie Rating,Cast_1,Cast_2,Cast_3,Director,Budget,Collection,Verdict
0,PK,An alien on Earth loses the only device he can...,8.1,Aamir Khan,Anushka Sharma,Sanjay Dutt,Rajkumar Hirani,[85],[792],All Time Blockbuster
1,Happy New Year,A gang of amateur dancers takes part in a danc...,5.0,Shah Rukh Khan,Deepika Padukone,Abhishek Bachchan,Farah Khan,[150],[397],Hit
2,Kick,An adrenaline junkie walks away from a whirlwi...,6.0,Salman Khan,Jacqueline Fernandez,Nawazuddin Siddiqui,Sajid Nadiadwala,[140],[378],Super Hit
3,Bang Bang,A young bank receptionist gets mixed up with R...,5.6,Hrithik Roshan,Katrina Kaif,Pawan Malhotra,Siddharth Anand,[140],[340],Super Hit
4,Singham Returns,Owing to the wrongdoings affiliated with evils...,5.7,Ajay Devgn,Kareena Kapoor,Amole Gupte,Rohit Shetty,[80],[216],Super Hit
...,...,...,...,...,...,...,...,...,...,...
668,NTR Kathanayakudu,Based on the journey of the legendary Telugu a...,5.8,Nandamuri Balakrishna,Vidya Balan,Kaikala Satyanarayana,Radha Krishna Jagarlamudi,[1],[],Hit
669,Love Action Drama,A man goes through all kinds of action to impr...,5.2,Nivin Pauly,Nayanthara,Aju Varghese,Dhyan Sreenivasan,[1],[6],Super Hit
670,Driving Licence,A famous actor needs to renew his driver's lic...,7.2,Prithviraj Sukumaran,Suraj Venjaramoodu,Miya George,Jean Paul Lal,[7],[1],Disaster
671,Mr. Local,A happy-go-lucky guy lands in trouble after he...,3.3,Sivakarthikeyan,Nayanthara,Radhika Sarathkumar,M. Rajesh,[3],[4],Blockbuster


In [11]:
df.to_excel('Movie_Performance_Data.xlsx', index=False)